In [ ]:
import os
import cv2
import shutil
# import zipfile
from tqdm import tqdm
from os.path import join
import numpy as np
import imgaug.augmenters as iaa
from pathlib import Path


seq = iaa.Sequential([
      iaa.Crop(px=(0, 8), keep_size=True), #Dla obrazka 64px więcej niż 8 to dużo
      iaa.Fliplr(0.5),
      iaa.GaussianBlur(sigma=(0, 1.0)), #Większy blur polepszy uogólnianie?
      iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),

      iaa.SimplexNoiseAlpha(iaa.OneOf([
        iaa.EdgeDetect(alpha=(0.0, .3)),
        iaa.DirectedEdgeDetect(alpha=(0.0, .5), direction=(0.0, 0.5)),
      ])),

      iaa.Emboss(alpha=(0, 1.0), strength=(0, 0.4)), # Taka maska wyciagajaca fakturę
      iaa.Dropout((0.01, 0.1), per_channel=0.5), #Robi czarne kropki
      # iaa.CoarseDropout((0.03, 0.15), size_percent=(0.01, 0.03), per_channel=0.2), #Robi czarny kwadrat
      iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
      iaa.PiecewiseAffine(scale=(0.01, 0.05)), #Lekkie zniekształcenia
      iaa.PerspectiveTransform(scale=(0.01, 0.1)),

      iaa.Add((-10, 10), per_channel=0.5), #Brightness
      iaa.FrequencyNoiseAlpha(
        exponent=(-1, 0),
        first=iaa.Multiply((0.7, 1.3), per_channel=True),
        second=iaa.LinearContrast((0.8, 1.3))
      ),
  ])

train_path = "train_v1-100k"
test_path = "test_v1-100k"
Path(train_path).mkdir(parents=True, exist_ok=True)
Path(test_path).mkdir(parents=True, exist_ok=True)

dataset_resized_count = len(os.listdir("dataset_resized"))  # ok 13_000 jest
desired_count = 100_000
i = 0

while desired_count > i:
    for image in tqdm(os.listdir("dataset_resized")):
        img = cv2.imread(join("dataset_resized", image))
        imgs = np.array([img])
        image_aug = seq(images=imgs)[0]

        image_split = image.split('.')
        filename = image_split[0] + f"_augid{i}." + image_split[1]
        train_gray = cv2.cvtColor(image_aug, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(join(train_path, filename), train_gray)

        test_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        cv2.imwrite(join(test_path, filename), test_gray)

        i += 1
        if i > desired_count:
            break